# Automatic Keyword Extraction With RAKE

### RAKE คืออะไร?
>RAKE ย่อมาจาก rapid automatic keyword extraction เป็น library ภาษาไพธอนที่อิมพลีเมนต์ตามอัลกอริทึม Rapid Automatic Keyword Extraction (RAKE) ในงานวิจัย Automatic Keyword Extraction from Individual Documents. ของ Rose, S., Engel, D., Cramer, N., & Cowley, W. (2010). นอกจากนี้ไลบรารี่ยังมีเวอร์ชั่นปรับปรุงที่เปลี่ยนไปใช้ NLTK สำหรับการคำนวนแทนการใช้ความสามารถของภาษาไพธอนตามปรกติ 

### Keyword Extraction ทำงานอย่างไร

1. __Candidate selection__ : ในขั้นนี้เราจะแกะเอาคำที่เป็นไปได้ทั้งหมดไม่ว่าจะเป็น วลี เทอม หรือ คอนเซปต์(ขึ้นอยู๋กับงาน) ที่มีศักยภาพที่จะเป็นคำสำคัญได้(Keyword)

2. __Properties calculation__ : สำหรับ candidate แต่ละตัวที่หาได้ในขั้นตอนก่อนหน้า เราจะหาคุณสมบัติที่บ่งบอกว่า candidate ตัวนี้จะเป็นคำสำคัญ ยกตัวอย่างเช่น candidate ที่ปรากฎอยู่ใน title ของหนังสือมีโอกาสสูงที่จะเป็นคำสำคัญ เป็นต้น

3. __Scoring and selecting keywords__ : candidate แต่ละตัวสามารถให้คะแนนได้โดยรวมคุณสมบัติต่างๆ เข้าด้วยกันโดยใช้สูตรหรือใช้เทคนิคการเรียนรู้ของเครื่องในการคำนวนค่าความน่าจะเป็นของแต่ละ candidate ว่าจะเป็นคำสำคัญได้หรือไม่ หลังจากนั้นจะเรียงลำดับ candidate แต่ละตัวตามค่าคะแนนหรือค่าความน่าจะเป็นของแต่ละตัว แล้วเลือก candidate ที่คะแนนหรือความน่าจะเป็นมากมาเป็นคำสำคัญ

4. __Improvement__ : สุดท้ายพารามิเตอร์ เช่น ความถี่ต่ำสุดของแต่ละ candidate, ค่าความยาวมากสุดและน้อยสุดในแต่ละคำ, หรือการใช้ stemmer ในการ normalize candidate แต่ละตัวจะช่วยในการปรับปรุงประสิทธิภาพของอัลกอริธึมในแต่ละดาต้าเซต

In [4]:
from __future__ import absolute_import
from __future__ import print_function
import rake
import operator
import six
import io

อันดับแรกเราจะต้องสร้าง rake object และบอกพาธที่อยู่ของ stopword เป็นพารามิเตอร์ โดยภายในไฟล์ stopword เป็นลิสต์ของคำที่เรานิยามให้เป็น stopword ดูนิยามของ stopword ได้ที่นี่ __[กด](https://en.wikipedia.org/wiki/Stop_words)__

In [5]:
rake_object = rake.Rake("SmartStoplist.txt")

In [6]:
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility " \
       "of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. " \
       "Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating"\
       " sets of solutions for all types of systems are given. These criteria and the corresponding algorithms " \
       "for constructing a minimal supporting set of solutions can be used in solving all the considered types of " \
       "systems and systems of mixed types."

ต่อมาเราจะใช้คำสั่ง run ของ rake_object และส่งค่าพารามิเตอร์เป็น text ที่ต้องการให้แกะ keyword โดยการทำงานเบื้องหลังฟังก์ชั่น run ประกอบด้วยขั้นตอนดังนี้

1. นำ text ที่รับพารามิเตอร์มาผ่านฟังก์ชั่น __split_sentences()__ เพิ่อแบ่ง text ที่ได้รับมาออกเป็นประโยค โดยฟังก์ชั่นจะคืนค่าออกมาเป็นลิสต์ที่ประกอบด้วยประโยคที่ตัดแล้ว(sentence_list) 

2. อ่านไฟล์ SmartStoplist.txt แล้วนำไปสร้างเป็น stopword_list จากนั้นนำ stopword_list ผ่านฟังก์ชั่น __build_stop_word_regex()__ สำหรับใช้ค้นหา stopword ภายใน text ที่ต้องการแกะ Keyword โดยฟังก์ชั่นจะคืนค่าเป็น stop_words_pattern

    > __Regular Expressions__ หรือเรียกย่อๆว่า “ regex ” คือ การกำหนดรูปแบบหรือกลุ่มคำ เพื่อเอาไว้ใช้ค้นหาข้อความต่างๆตามที่เราต้องการ สามารถค้นหาได้ทั้งอักขระธรรมดา หรือค้นหาความข้อที่กำหนดไว้ หรือจะเป็นอักขระพิเศษก็สามารถค้นหาได้ 

3. นำ sentence_list และ stop_words_pattern ที่ได้จากขั้นตอนก่อนหน้าผ่านฟังก์ชั่น __generate_candidate_keywords()__ โดยฟังก์ชั่นจะคืนค่ามาเป็น phrase_list

4. นำ phrase_list ที่ได้จากขั้นตอนก่อนหน้าผ่านฟังก์ชั่น __calculate_word_scores()__ เพื่อคำนวนคะแนนของแต่ละ phrase โดยฟังก์ชั่นจะคืนค่าออกมาเป็น word_score

5. นำ word_score และ phrase_list ผ่านฟังก์ชั่น __generate_candidate_keyword_scores()__ เพื่อสร้าง dictionary ที่มี Key เป็น phrase และ Value เป็น Score โดยฟังก์ชั่นจำคืนค่าออกมาเป็น keyword_candidates

6. เรียงลำดับ keyword_candidates ตามคะแนนจากมากไปน้อย แล้วคืนค่ากลับ จบการทำงานของฟังก์ชั่น run

## หน้าตาฟังก์ชั่น run
``` python
def run(self, text):
        sentence_list = split_sentences(text)

        stop_words_pattern = build_stop_word_regex(self.__stop_words_list)

        phrase_list = generate_candidate_keywords(sentence_list, stop_words_pattern, self.__stop_words_list,
                                                  self.__min_char_length, self.__max_words_length,
                                                  self.__min_words_length_adj, self.__max_words_length_adj,
                                                  self.__min_phrase_freq_adj)

        word_scores = calculate_word_scores(phrase_list)

        keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores, self.__min_keyword_frequency)

        sorted_keywords = sorted(six.iteritems(keyword_candidates), key=operator.itemgetter(1), reverse=True)
        return sorted_keywords
```

### ทดลองใช้ฟังก์ชั่น run

In [21]:
keywords = rake_object.run(text)

for keyword in keywords:
    print("Keyword : ", keyword[0], " | Score : {:.2f}".format(keyword[1]))

Keyword :  minimal generating sets  | Score : 8.67
Keyword :  linear diophantine equations  | Score : 8.50
Keyword :  minimal supporting set  | Score : 7.67
Keyword :  minimal set  | Score : 4.67
Keyword :  linear constraints  | Score : 4.50
Keyword :  natural numbers  | Score : 4.00
Keyword :  strict inequations  | Score : 4.00
Keyword :  nonstrict inequations  | Score : 4.00
Keyword :  upper bounds  | Score : 4.00
Keyword :  mixed types  | Score : 3.67
Keyword :  considered types  | Score : 3.17
Keyword :  set  | Score : 2.00
Keyword :  types  | Score : 1.67
Keyword :  considered  | Score : 1.50
Keyword :  compatibility  | Score : 1.00
Keyword :  systems  | Score : 1.00
Keyword :  criteria  | Score : 1.00
Keyword :  system  | Score : 1.00
Keyword :  components  | Score : 1.00
Keyword :  solutions  | Score : 1.00
Keyword :  algorithms  | Score : 1.00
Keyword :  construction  | Score : 1.00
Keyword :  constructing  | Score : 1.00
Keyword :  solving  | Score : 1.00


***
## ที่มา

__[zelandiya/RAKE-tutorial](https://github.com/zelandiya/RAKE-tutorial)__ <br>
__[Intro to Automatic Keyword Extraction](https://medium.com/@Chitly/automatic-keyword-extraction-part-1-63bf7a057cb5)__ <br>
__[NLP keyword extraction tutorial with RAKE and Maui](https://www.airpair.com/nlp/keyword-extraction-tutorial)__